In [547]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot  as plt
%matplotlib inline

In [548]:
path = os.getcwd() 
data_root = path + '\\DATA\\'
data_root

'C:\\Users\\Kaipable\\Desktop\\GitHub\\voter-protection-corps\\pollworkers_src\\DATA\\'

In [549]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [550]:
#Read In
all_df = pd.read_csv(data_root+'all_states_risk_data.csv')

### Customize Metrics and  Indicators
Define Your Categories, Variables, and Abbreviations Below in metric_map

In [551]:
metric_map = {
    'priority' : ('PRI',['Non-white Residents','Senior Residents']),
    'recruiting risk' : ('REC',['Senior Poll Workers','Reported Difficulty ']),
     'demand risk' : ('D',['In-Person Voters', 'Inactive Voters']),
    'resource supply' : ('S',['Voters Per Location','Voters Per Machine','Voters Per Pollworker','Wait Times']),
}
the_metrics = [i for i in metric_map.keys()]
#the_indicators = [ for k in the_metrics for acronym, indicator_list in metric_map[k]

agg_metrics = ('mean', 'median', 'max', 'min')

In [552]:
def get_percentiles(df):
    for i in metric_map.keys():
        for j in metric_map[i][1]:
            keep_series=df.groupby("State_Abbr")[j].count() >0
            viable_states = keep_series[keep_series==True].index
            temp_df = df[df['State_Abbr'].isin(viable_states)]
            intermed= temp_df.groupby("State_Abbr")[j].rank(pct=True, na_option="keep")
            df["("+metric_map[i][0]+") "+j] = intermed
    return df
        
def get_agg_columns(df):
    funct_map = {'median': np.nanmedian,
            'mean': np.nanmean, 
            'min': np.min,
            'max': np.max}
    for metric in metric_map.keys():
        percent_list = ["("+metric_map[metric][0]+") " + j for j in metric_map[metric][1]] 
        df[metric.title()+" ("+agg_method.title()+")"] = funct_map[agg_method](df[percent_list], axis=1)
        if agg_method not in funct_map.keys():
            print('Please select an acceptable metric: "min", "max", "mean", or "median".')
    return df

def get_final_scores(df):
    for i in metric_map.keys():
        keep_series=df.groupby("State_Abbr")[i.title()+" ("+agg_method.title()+")"].count() >0
        viable_states = keep_series[keep_series==True].index
        temp_df = df[df['State_Abbr'].isin(viable_states)]
        rel_scores = temp_df.groupby("State_Abbr")[i.title()+" ("+agg_method.title()+")"].rank(pct=True, na_option="keep")
        rel_heat = rel_scores.apply(lambda x: "High" if x > 0.75  else ("Low" if x <0.5 else (np.nan if np.isnan(x) else "Medium")))
        df[i.title()+" (Final)"] = rel_scores
        df[i.title()+" (LMH)"]= rel_heat
    return df

def get_final_df(df, metric_map, agg_method="median"):
    agg_method = agg_method
    func_list = [get_percentiles, get_agg_columns, get_final_scores]
    for i in func_list:
        df = i(df)
    return final_df

In [553]:
scored_df = get_final_df(all_df, 'median', metric_map)

In [554]:
#Test with Arizona
scored_df[scored_df['State_Abbr']=="AZ"]

,Unnamed: 0,Full State,short_fips,State_Abbr,County Name,Non-white Population,Senior Residents,Senior Poll Workers,Reported Difficulty,In-Person Voters,Inactive Voters,Voters Per Location,Voters Per Machine,Voters Per Pollworker,Wait Times,Wait Count,# of Same Day Locations,# of Machines,# of Poll Workers,# of Senior Poll Workers,# of Voters Removed,# of Precincts,# Mail In Ballots,% Early Vote,% Rejected Ballots,Median Age,Median Household Income,Total Population,Total Eligible Voters,% Senior Poll Worker,% Votes in Person,% Inactive Voters,%Nonwhite,% Residents Over 60,(PRI) Non-white Population,(PRI) Senior Residents,(D) In-Person Voters,(D) Inactive Voters,(S) Voters Per Location,(S) Voters Per Machine,(S) Voters Per Pollworker,(S) Wait Times,(REC) Senior Poll Workers,(REC) Reported Difficulty,Priority Population (Median),Demand (Median),Supply (Median),Poll Worker Recruiting (Median),Priority Population (Final),Priority Population (LMH),Demand (Final),Demand (LMH),Supply (Final),Supply (LMH),Poll Worker Recruiting (Final),Poll Worker Recruiting (LMH),concat
69,69,ARIZONA,4001,AZ,APACHE COUNTY,55160.0,14758.0,99.0,4.0,13440.0,4971.0,1306.666667,1193.043478,197.410072,0.0,0.0,42.0,46.0,278.0,99.0,4930.0,44.0,-88.0,0.034939,0.017120,34.7,32963.0,71522.0,54880.0,0.356115,0.525842,0.090579,0.771231,0.206342,0.733333,0.333333,0.533333,0.333333,0.200000,0.400000,0.133333,0.533333,0.400000,0.466667,0.533333,0.433333,0.300000,0.433333,0.466667,Medium,0.400000,Medium,0.200000,Low,0.333333,Medium,MediumMediumHighHigh
70,70,ARIZONA,4003,AZ,COCHISE COUNTY,19036.0,36129.0,77.0,4.0,8591.0,16381.0,5131.470588,545.218750,510.146199,0.0,0.0,17.0,160.0,171.0,77.0,8539.0,49.0,34144.0,0.015873,0.016885,40.6,48649.0,126279.0,87235.0,0.450292,0.187058,0.187780,0.150746,0.286105,0.466667,0.600000,0.400000,0.600000,0.800000,0.133333,0.800000,0.533333,0.333333,0.466667,0.533333,0.500000,0.666667,0.400000,0.466667,Medium,0.466667,Medium,0.800000,High,0.266667,Medium,MediumMediumMediumMedium
71,71,ARIZONA,4005,AZ,COCONINO COUNTY,50715.0,24202.0,161.0,4.0,15391.0,15500.0,1795.254545,836.771186,281.307692,0.0,0.0,55.0,118.0,351.0,161.0,15114.0,71.0,37112.0,0.032780,0.005270,30.6,57616.0,140217.0,98739.0,0.458689,0.275095,0.156980,0.361689,0.172604,0.600000,0.466667,0.666667,0.533333,0.333333,0.266667,0.333333,0.533333,0.566667,0.466667,0.533333,0.600000,0.333333,0.516667,0.466667,Medium,0.600000,Medium,0.266667,Medium,0.533333,Medium,MediumMediumMediumMedium
72,72,ARIZONA,4007,AZ,GILA COUNTY,12305.0,19272.0,161.0,5.0,3635.0,7433.0,2178.470588,1157.312500,356.096154,0.0,0.0,17.0,32.0,104.0,161.0,4539.0,39.0,17144.0,0.037311,0.025941,49.7,42092.0,53400.0,37034.0,1.548077,0.174550,0.200707,0.230431,0.360899,0.333333,0.400000,0.333333,0.400000,0.466667,0.333333,0.600000,0.533333,0.566667,0.900000,0.366667,0.366667,0.500000,0.733333,0.333333,Medium,0.333333,Medium,0.400000,Medium,0.933333,High,MediumHighMediumMedium
73,73,ARIZONA,4009,AZ,GRAHAM COUNTY,7962.0,7083.0,25.0,3.0,3469.0,3250.0,2350.666667,1511.142857,480.818182,0.0,0.0,9.0,14.0,44.0,25.0,2903.0,22.0,7204.0,0.097627,0.015295,33.3,51352.0,37879.0,21156.0,0.568182,0.321621,0.153621,0.210196,0.186990,0.266667,0.133333,0.266667,0.266667,0.600000,0.533333,0.733333,0.533333,0.133333,0.133333,0.200000,0.266667,0.566667,0.133333,0.200000,Low,0.266667,Medium,0.533333,Medium,0.066667,Low,LowLowLowLow
74,74,ARIZONA,4011,AZ,GREENLEE COUNTY,862.0,1698.0,0.0,5.0,609.0,510.0,1009.000000,210.208333,180.178571,0.0,0.0,5.0,24.0,28.0,0.0,872.0,8.0,1905.0,0.732205,0.031195,34.2,60962.0,9504.0,5045.0,0.000000,0.234321,0.101090,0.090699,0.178662,0.066667,0.066667,0.066667,0.066667,0.133333,0.066667,0.066667,0.533333,0.066667,0.900000,0.066667,0.066667,0.100000,0.483333,0.066667,Low,0.066667,Low,0.066667,Low,0.400000,Medium,LowMediumLowLow
75,75,ARIZONA,4012,AZ,LA PAZ COUNTY,4787.0,9337.0,40.0,4.0,1729.0,1966.0,1501.750000,750.875000,203.627119,0.0,0.0,8.0,16.0,59.0,40.0,1678.0,11.0,3367.0,0.029822,0.004959,56.0,33333.

In [555]:
scored_df['Concat']= scored_df['Priority Population (LMH)'] + scored_df['Poll Worker Recruiting (LMH)'] + df['Resource Supply (LMH)'] + df['Resource Supply (LMH)']

In [556]:
scored_df.head()

,Unnamed: 0,Full State,short_fips,State_Abbr,County Name,Non-white Population,Senior Residents,Senior Poll Workers,Reported Difficulty,In-Person Voters,Inactive Voters,Voters Per Location,Voters Per Machine,Voters Per Pollworker,Wait Times,Wait Count,# of Same Day Locations,# of Machines,# of Poll Workers,# of Senior Poll Workers,# of Voters Removed,# of Precincts,# Mail In Ballots,% Early Vote,% Rejected Ballots,Median Age,Median Household Income,Total Population,Total Eligible Voters,% Senior Poll Worker,% Votes in Person,% Inactive Voters,%Nonwhite,% Residents Over 60,(PRI) Non-white Population,(PRI) Senior Residents,(D) In-Person Voters,(D) Inactive Voters,(S) Voters Per Location,(S) Voters Per Machine,(S) Voters Per Pollworker,(S) Wait Times,(REC) Senior Poll Workers,(REC) Reported Difficulty,Priority Population (Median),Demand (Median),Supply (Median),Poll Worker Recruiting (Median),Priority Population (Final),Priority Population (LMH),Demand (Final),Demand (LMH),Supply (Final),Supply (LMH),Poll Worker Recruiting (Final),Poll Worker Recruiting (LMH),concat,Concat
0,0,ALABAMA,1001,AL,AUTAUGA COUNTY,12763.0,10744.0,96.0,4.0,19081.0,3379.0,2093.157895,864.565217,337.033898,0.0,0.0,19.0,46.0,118.0,96.0,2678.0,19.0,728.0,0.0,0.005195,37.8,58786.0,55200.0,39770.0,0.813559,0.960147,0.084964,0.231214,0.194638,0.656716,0.597015,0.656716,0.686567,0.805970,0.791045,0.910448,0.5,0.477612,0.567164,0.626866,0.671642,0.798507,0.522388,0.671642,Medium,0.679104,Medium,0.865672,High,0.507463,Medium,MediumMediumHighHigh,MediumMediumHighHigh
1,1,ALABAMA,1003,AL,BALDWIN COUNTY,28581.0,55516.0,424.0,3.0,75068.0,13826.0,3165.857143,1251.024194,300.052224,0.0,0.0,49.0,124.0,517.0,424.0,14145.0,91.0,2748.0,0.0,0.007122,42.8,55962.0,208107.0,155127.0,0.820116,0.959164,0.089127,0.137338,0.266767,0.820896,0.955224,0.940299,0.955224,0.940299,0.955224,0.865672,0.5,0.940299,0.231343,0.888060,0.947761,0.902985,0.585821,0.895522,High,0.955224,High,0.970149,High,0.656716,Medium,HighMediumHighHigh,HighMediumHighHigh
2,2,ALABAMA,1005,AL,BARBOUR COUNTY,13566.0,6285.0,72.0,4.0,7690.0,1452.0,184.287234,524.939394,190.362637,0.0,0.0,94.0,33.0,91.0,72.0,1409.0,21.0,684.0,0.0,0.095662,39.9,34186.0,25782.0,17323.0,0.791209,0.909521,0.083819,0.526181,0.243775,0.686567,0.373134,0.343284,0.373134,0.029851,0.597015,0.597015,0.5,0.343284,0.567164,0.529851,0.358209,0.548507,0.455224,0.567164,Medium,0.373134,Medium,0.537313,Medium,0.417910,Medium,MediumMediumMediumMedium,MediumMediumMediumMedium
3,3,ALABAMA,1007,AL,BIBB COUNTY,5259.0,4877.0,52.0,3.0,6668.0,999.0,1780.500000,749.684211,200.619718,0.0,0.0,8.0,19.0,71.0,52.0,886.0,16.0,196.0,0.0,0.000000,39.9,45340.0,22527.0,14244.0,0.732394,0.970597,0.070135,0.233453,0.216496,0.253731,0.283582,0.238806,0.238806,0.716418,0.716418,0.626866,0.5,0.231343,0.231343,0.268657,0.238806,0.671642,0.231343,0.268657,Medium,0.238806,Low,0.716418,Medium,0.104478,Low,MediumLowMediumMedium,MediumLowMediumMedium
4,4,ALABAMA,1009,AL,BLOUNT COUNTY,2591.0,13846.0,104.0,4.0,19120.0,2335.0,1526.423077,763.211538,317.496000,0.0,0.0,26.0,52.0,125.0,104.0,2350.0,29.0,426.0,0.0,0.002232,40.8,48695.0,57645.0,39687.0,0.832000,0.977405,0.058835,0.044948,0.240194,0.119403,0.671642,0.671642,0.507463,0.656716,0.731343,0.895522,0.5,0.537313,0.567164,0.395522,0.589552,0.694030,0.552239,0.388060,Medium,0.574627,Medium,0.746269,Medium,0.559701,Medium,MediumMediumMediumMedium,MediumMediumMediumMedium


Fill out decision matrix in csv to read in here as "action_map.csv"

In [557]:
# Read in Action Decision Matrix
actions = pd.read_csv('action_map.csv')
actions.columns
actions.head()

,Priority Population,Recruiting Difficulty,Demand Risk,Resource Supply,Concat,Action
0,Low,Low,Low,Low,LowLowLowLow,Monitor
1,Low,Low,Low,Medium,LowLowLowMedium,Recruit
2,Low,Low,Low,High,LowLowLowHigh,Recruit
3,Low,Low,Medium,Low,LowLowMediumLow,Monitor
4,Low,Low,Medium,Medium,LowLowMediumMedium,Recruit


In [560]:
final_df = pd.merge(scored_df,actions[['Concat', 'Action']], on='Concat', how='left')

In [561]:
final_df.head()

,Unnamed: 0,Full State,short_fips,State_Abbr,County Name,Non-white Population,Senior Residents,Senior Poll Workers,Reported Difficulty,In-Person Voters,Inactive Voters,Voters Per Location,Voters Per Machine,Voters Per Pollworker,Wait Times,Wait Count,# of Same Day Locations,# of Machines,# of Poll Workers,# of Senior Poll Workers,# of Voters Removed,# of Precincts,# Mail In Ballots,% Early Vote,% Rejected Ballots,Median Age,Median Household Income,Total Population,Total Eligible Voters,% Senior Poll Worker,% Votes in Person,% Inactive Voters,%Nonwhite,% Residents Over 60,(PRI) Non-white Population,(PRI) Senior Residents,(D) In-Person Voters,(D) Inactive Voters,(S) Voters Per Location,(S) Voters Per Machine,(S) Voters Per Pollworker,(S) Wait Times,(REC) Senior Poll Workers,(REC) Reported Difficulty,Priority Population (Median),Demand (Median),Supply (Median),Poll Worker Recruiting (Median),Priority Population (Final),Priority Population (LMH),Demand (Final),Demand (LMH),Supply (Final),Supply (LMH),Poll Worker Recruiting (Final),Poll Worker Recruiting (LMH),concat,Concat,Action
0,0,ALABAMA,1001,AL,AUTAUGA COUNTY,12763.0,10744.0,96.0,4.0,19081.0,3379.0,2093.157895,864.565217,337.033898,0.0,0.0,19.0,46.0,118.0,96.0,2678.0,19.0,728.0,0.0,0.005195,37.8,58786.0,55200.0,39770.0,0.813559,0.960147,0.084964,0.231214,0.194638,0.656716,0.597015,0.656716,0.686567,0.805970,0.791045,0.910448,0.5,0.477612,0.567164,0.626866,0.671642,0.798507,0.522388,0.671642,Medium,0.679104,Medium,0.865672,High,0.507463,Medium,MediumMediumHighHigh,MediumMediumHighHigh,Recruit Immediately
1,1,ALABAMA,1003,AL,BALDWIN COUNTY,28581.0,55516.0,424.0,3.0,75068.0,13826.0,3165.857143,1251.024194,300.052224,0.0,0.0,49.0,124.0,517.0,424.0,14145.0,91.0,2748.0,0.0,0.007122,42.8,55962.0,208107.0,155127.0,0.820116,0.959164,0.089127,0.137338,0.266767,0.820896,0.955224,0.940299,0.955224,0.940299,0.955224,0.865672,0.5,0.940299,0.231343,0.888060,0.947761,0.902985,0.585821,0.895522,High,0.955224,High,0.970149,High,0.656716,Medium,HighMediumHighHigh,HighMediumHighHigh,Recruit Immediately
2,2,ALABAMA,1005,AL,BARBOUR COUNTY,13566.0,6285.0,72.0,4.0,7690.0,1452.0,184.287234,524.939394,190.362637,0.0,0.0,94.0,33.0,91.0,72.0,1409.0,21.0,684.0,0.0,0.095662,39.9,34186.0,25782.0,17323.0,0.791209,0.909521,0.083819,0.526181,0.243775,0.686567,0.373134,0.343284,0.373134,0.029851,0.597015,0.597015,0.5,0.343284,0.567164,0.529851,0.358209,0.548507,0.455224,0.567164,Medium,0.373134,Medium,0.537313,Medium,0.417910,Medium,MediumMediumMediumMedium,MediumMediumMediumMedium,Recruit Immediately
3,3,ALABAMA,1007,AL,BIBB COUNTY,5259.0,4877.0,52.0,3.0,6668.0,999.0,1780.500000,749.684211,200.619718,0.0,0.0,8.0,19.0,71.0,52.0,886.0,16.0,196.0,0.0,0.000000,39.9,45340.0,22527.0,14244.0,0.732394,0.970597,0.070135,0.233453,0.216496,0.253731,0.283582,0.238806,0.238806,0.716418,0.716418,0.626866,0.5,0.231343,0.231343,0.268657,0.238806,0.671642,0.231343,0.268657,Medium,0.238806,Low,0.716418,Medium,0.104478,Low,MediumLowMediumMedium,MediumLowMediumMedium,Recruit
4,4,ALABAMA,1009,AL,BLOUNT COUNTY,2591.0,13846.0,104.0,4.0,19120.0,2335.0,1526.423077,763.211538,317.496000,0.0,0.0,26.0,52.0,125.0,104.0,2350.0,29.0,426.0,0.0,0.002232,40.8,48695.0,57645.0,39687.0,0.832000,0.977405,0.058835,0.044948,0.240194,0.119403,0.671642,0.671642,0.507463,0.656716,0.731343,0.895522,0.5,0.537313,0.567164,0.395522,0.589552,0.694030,0.552239,0.388060,Medium,0.574627,Medium,0.746269,Medium,0.559701,Medium,MediumMediumMediumMedium,MediumMediumMediumMedium,Recruit Immediately


In [ ]:
final_df.to_csv('all_states_risk_scores.csv')